In [ ]:
!pip install findspark
import findspark
findspark.init()
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
if __name__ == "__main__":
    sc = SparkContext(appName="trail")
    sc.setLogLevel("WARN")
    ss=SparkSession(sc)
    ssc = StreamingContext(sc,360)
    modelpath="E:\model"
    rfmodel=RandomForestClassificationModel.load(modelpath)
	#localhost:2181 = Default Zookeeper Consumer Address
    kafkaStream = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'hrtdis':1})
    
    parsed = kafkaStream.map(lambda v: json.loads(v[1]))
    txt=parsed.map(lambda x: x['text']).map(lambda x: x.split())
    def f(x):
        try:
            df=ss.createDataFrame(x,['h1','h2','AGE','SEX','CPT','RBP','SCH','FBS','RES','MHR','EIA','OPK','PES','VCA','THA'])
            df=df.drop('h1','h2')
            df=df.withColumn('AGE',df['AGE'].cast(DoubleType()))
            df=df.withColumn('SEX',df['SEX'].cast(DoubleType()))
            df=df.withColumn('CPT',df['CPT'].cast(DoubleType()))
            df=df.withColumn('RBP',df['RBP'].cast(DoubleType()))
            df=df.withColumn('SCH',df['SCH'].cast(DoubleType()))
            df=df.withColumn('FBS',df['FBS'].cast(DoubleType()))
            df=df.withColumn('RES',df['RES'].cast(DoubleType()))
            df=df.withColumn('MHR',df['MHR'].cast(DoubleType()))
            df=df.withColumn('EIA',df['EIA'].cast(DoubleType()))
            df=df.withColumn('OPK',df['OPK'].cast(DoubleType()))
            df=df.withColumn('PES',df['PES'].cast(DoubleType()))
            df=df.withColumn('VCA',df['VCA'].cast(DoubleType()))
            df=df.withColumn('THA',df['THA'].cast(DoubleType()))
            df.show()
            cols=df.columns
            assembler=VectorAssembler(inputCols=cols,outputCol='featuresvector')
            df=assembler.transform(df)
            df.select('featuresvector').show()
            scaler=MinMaxScaler(inputCol='featuresvector',outputCol='scaledfeatures')
            scalermodel=scaler.fit(df)
            df=scalermodel.transform(df)
            df.select("scaledfeatures").show()
            df=rfmodel.transform(df)
            df.select('prediction').show()
        except:
            print('no patient data available')
    txt.foreachRDD(f)
    ssc.start()
    ssc.awaitTermination()

You should consider upgrading via the 'c:\users\mekal\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


+----+---+---+-----+-----+---+---+-----+---+---+---+---+---+
| AGE|SEX|CPT|  RBP|  SCH|FBS|RES|  MHR|EIA|OPK|PES|VCA|THA|
+----+---+---+-----+-----+---+---+-----+---+---+---+---+---+
|37.0|1.0|4.0|194.0|292.0|0.0|2.0|144.0|0.0|0.0|1.0|1.0|2.0|
|40.0|1.0|1.0|114.0|282.0|1.0|2.0|114.0|0.0|2.0|1.0|1.0|7.0|
|67.0|1.0|2.0|138.0|255.0|0.0|2.0|152.0|1.0|0.0|1.6|0.0|3.0|
|57.0|1.0|3.0|154.0|196.0|0.0|2.0|288.0|0.0|0.0|1.0|1.0|7.0|
+----+---+---+-----+-----+---+---+-----+---+---+---+---+---+

+--------------------+
|      featuresvector|
+--------------------+
|[37.0,1.0,4.0,194...|
|[40.0,1.0,1.0,114...|
|[67.0,1.0,2.0,138...|
|[57.0,1.0,3.0,154...|
+--------------------+

+--------------------+
|      scaledfeatures|
+--------------------+
|[0.0,0.5,1.0,1.0,...|
|[0.1,0.5,0.0,0.0,...|
|[1.0,0.5,0.333333...|
|[0.66666666666666...|
+--------------------+

+----------+
|prediction|
+----------+
|       1.0|
|       0.0|
|       1.0|
|       1.0|
+----------+

no patient data available
no patient